# Exploracion datos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
import ydata_profiling 
import scipy.stats as stats

In [ ]:

df = pd.read_csv('./data/datos_apartamentos_rent_clean.csv', sep=';')

In [ ]:
ydata_profiling.ProfileReport(df)

In [ ]:
#Nombres de columnas
df.columns = df.columns.str.strip()

df.columns = df.columns.str.replace(',', '', regex=True)  
df = df.rename(columns={df.columns[-1]: "time"})  


df['square_feet'] = pd.to_numeric(df['square_feet'], errors='coerce')


print(df.columns.tolist())


In [ ]:
print(df.info())

In [ ]:
# datos nulos 
print(df.isnull().sum())

In [ ]:
# cambiar la categoria de las variables 

df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['square_feet'] = pd.to_numeric(df['square_feet'], errors='coerce')
df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce')
df['bedrooms'] = pd.to_numeric(df['bedrooms'], errors='coerce')


print(df[['price', 'square_feet', 'bathrooms', 'bedrooms']].info())

In [ ]:
# Estadísticas generales de las variables numéricas
print(df.describe())

In [ ]:
# Histograma de variables numericas
%matplotlib inline
numeric_columns = ['bathrooms', 'bedrooms', 'price', 'square_feet', 'longitude']

df_clean = df.dropna(subset=numeric_columns)

for col in numeric_columns:
    plt.figure(figsize=(8, 5))  
    sb.histplot(df_clean[col], bins=50, kde=True, color='blue')  
    plt.title(f'Histograma de {col}')
    plt.xlabel(col)
    plt.ylabel('Frecuencia')
    plt.grid(True)
    plt.show()  

In [ ]:
# Ver estadística de variables categóricas
print(df.describe(include=['O']))

In [ ]:
# grafica de barras de variables categoricas
categorical_columns = ['category', 'currency', 'fee', 'has_photo', 'pets_allowed', 'state', 'source']

for col in categorical_columns:
    plt.figure(figsize=(10, 10))
    sb.countplot(y=df[col], order=df[col].value_counts().index, palette='coolwarm')
    plt.title(f'Distribución de {col}')
    plt.xlabel('Frecuencia')
    plt.ylabel(col)
    plt.grid(True)
    plt.show()


In [ ]:
# Boxplot variables numéricas
numeric_columns = ['bathrooms', 'bedrooms', 'price', 'square_feet', 'longitude']

for col in numeric_columns:
    plt.figure(figsize=(8, 5))  
    sb.boxplot(x=df[col], color='blue')  
    plt.title(f'Boxplot de {col}')
    plt.xlabel(col)
    plt.grid(True)
    plt.show()


In [ ]:
#Graficos de Dispercion 

#Precio-metros cuadrados
plt.figure(figsize=(8, 5))
sb.scatterplot(x=df['square_feet'], y=df['price'], alpha=0.5)
plt.title('Precio vs. Tamaño (square_feet)')
plt.xlabel('Tamaño (square_feet)')
plt.ylabel('Precio')
plt.grid(True)
plt.show()

#Precio-num habitaciones
plt.figure(figsize=(8, 5))
sb.scatterplot(x=df['bedrooms'], y=df['price'], alpha=0.5)
plt.title('Precio vs. Número de Habitaciones')
plt.xlabel('Número de Habitaciones')
plt.ylabel('Precio')
plt.grid(True)
plt.show()

#Precio-num baños
plt.figure(figsize=(8, 5))
sb.scatterplot(x=df['bathrooms'], y=df['price'], alpha=0.5)
plt.title('Precio vs. Número de Baños')
plt.xlabel('Número de Baños')
plt.ylabel('Precio')
plt.grid(True)
plt.show()

#Precio-longitud
plt.figure(figsize=(8, 5))
sb.scatterplot(x=df['longitude'], y=df['price'], alpha=0.5)
plt.title('Precio vs. Ubicación (Longitud)')
plt.xlabel('Longitud')
plt.ylabel('Precio')
plt.grid(True)
plt.show()

In [ ]:
#Graficos de Violin 

#precio-habitaciones
plt.figure(figsize=(8, 5))
sb.violinplot(x=df['bedrooms'], y=df['price'], inner='quartile')
plt.title('Distribución del Precio por Número de Habitaciones')
plt.xlabel('Número de Habitaciones')
plt.ylabel('Precio')
plt.grid(True)
plt.show()

#baños-precio
plt.figure(figsize=(8, 5))
sb.violinplot(x=df['bathrooms'], y=df['price'], inner='quartile')
plt.title('Distribución del Precio por Número de Baños')
plt.xlabel('Número de Baños')
plt.ylabel('Precio')
plt.grid(True)
plt.show()


In [ ]:
#linealidad de datos cuantitativos
df[['price', 'square_feet', 'bathrooms', 'bedrooms']].corr()



# independencia
X = df[['square_feet', 'bathrooms', 'bedrooms']]  
X = sm.add_constant(X)  # Agregar intercepto
y = df['price']  

X.replace([np.inf, -np.inf], np.nan, inplace=True)  
df_clean = X.join(y).dropna() 

X = df_clean.drop(columns=['price'])
y = df_clean['price']

modelo = sm.OLS(y, X).fit()

# Test Durbin-Watson
from statsmodels.stats.stattools import durbin_watson
residuals = modelo.resid
print("Durbin-Watson:", durbin_watson(residuals))


#Homocedasticidad
plt.scatter(modelo.fittedvalues, modelo.resid, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel("Valores Ajustados")
plt.ylabel("Residuos")
plt.title("Gráfico de Residuos vs Valores Ajustados")
plt.show()

#Normalidad

sb.histplot(modelo.resid, kde=True)
plt.title("Distribución de Residuos")
plt.show()

stats.probplot(modelo.resid, dist="norm", plot=plt)
plt.title("Gráfico Q-Q de Residuos")
plt.show()
